In [1]:
!pip install dropbox

In [5]:
import json
import dropbox
import pandas as pd
import pickle

In [2]:
token = "S8KbZcyyXAkAAAAAAAI6tgbmPXTlYglQAJXsbeIFM2saa8tOEu2MhFbo0q2JOMBZ"

In [3]:
dbx = dropbox.Dropbox(token)

In [4]:
entries = []
lf_res = None
while True:
    if lf_res is None:
        lf_res = dbx.files_list_folder("/whoscored")
    else:
        lf_res = dbx.files_list_folder_continue(lf_res.cursor)
    entries += lf_res.entries
    if not lf_res.has_more:
        break

In [5]:
with open("entries", "wb") as fp:
    pickle.dump(entries, fp)

In [32]:
with open("entries", "rb") as fp:
    entries = pickle.load(fp)

In [33]:
def dic_from_entry(en):
    return json.loads(dbx.files_download(en.path_lower)[1].content)

In [34]:
len(entries)

42687

In [12]:
def save_dics(entries):
    bins_start = 0
    bins_end = 999
    for bin_num in range(8,10):      
        list_of_dics = [dic_from_entry(entries[i]) for i in range(bins_start,bins_end)]
        with open("list_of_dics_{}".format(bin_num), "wb") as fp:
            pickle.dump(list_of_dics, fp)
        print(bins_start, bins_end)
        bins_start+=1000
        bins_end+=1000
        list_of_dics = []
        print("{} laps are finished".format(bin_num+1))

In [13]:
save_dics(entries)

0 999
9 laps are finished


In [6]:
with open("all_dics_8", "rb") as fp:
    list_of_dics = pickle.load(fp)

In [7]:
#checks for empty dictionaries and deletes them
for i in range(2):
    a = len(list_of_dics)

    for idx, i in enumerate(list_of_dics):
        if len(i.keys()) != 28:
            del list_of_dics[idx]
        else:
            pass
    print("{} dicts were deleted".format(a - len(list_of_dics)))

28 dicts were deleted
2 dicts were deleted


In [8]:
home_team_id_dic = {}
away_team_id_dic = {}

for i in list_of_dics:
    home_team_id_dic[i["home"]["teamId"]] = i["home"]['name']
    away_team_id_dic[i["away"]["teamId"]] = i["away"]['name']

team_id_dic = home_team_id_dic

for v in away_team_id_dic:
    try:
        if away_team_id_dic[v] != home_team_id_dic[v]:
            raise ValueError('TeamID and TeamName differs in the away_team_id_dic and home_team_id_dic at index {}'\
                             .format(v))
    except:
        pass
    if v in home_team_id_dic.keys():
        pass
    else:
        team_id_dic[v] = away_team_id_dic[v]

In [9]:
player_id_dic = {}
for i in list_of_dics:
    for player_key in i["playerIdNameDictionary"].keys():
        if player_key not in player_id_dic:
            player_id_dic[player_key] = i["playerIdNameDictionary"][player_key]            
    for away_player_key in i["away"]["players"]:
        if away_player_key["playerId"] not in player_id_dic:
            player_id_dic[away_player_key["playerId"]] = away_player_key["name"]

In [10]:
def database_maker(list_of_dics):
    touches = []
    for idx, i in enumerate(list_of_dics):
        for e in i["events"]:
            if e["type"]["displayName"] == 'Pass':
                try:
                    touches.append(
                        {                     
                        "home_team": team_id_dic[e["teamId"]],
                        "Player": player_id_dic[str(e["playerId"])],
                        "Date": i["startDate"][:4],
                        "start_x": e["x"],
                        "start_y": e["y"],
                        "end_x": e["endX"],                
                        "end_y": e["endY"],
                        "Outcome": e["outcomeType"]["displayName"],
                        "Minute": e["minute"]   
                        }
                    )

                except:
                    pass
    return touches

In [11]:
touches = database_maker(list_of_dics)

In [12]:
touch_df = pd.DataFrame(touches)

In [13]:
touch_df = touch_df.assign(Distance = lambda x: ((x.end_x - x.start_x)**2 + (x.end_y - x.start_y)**2)**(1/2))

In [14]:
touch_df

,Date,Minute,Outcome,Player,end_x,end_y,home_team,start_x,start_y,Distance
0,2019,0,Successful,Corey Baird,36.9,56.7,Salt Lake,50.0,49.7,14.852946
1,2019,0,Unsuccessful,Kyle Beckerman,68.0,12.3,Salt Lake,38.9,59.9,55.790411
2,2019,0,Successful,Victor Giro,26.6,76.7,Vancouver,41.2,91.9,21.076053
3,2019,0,Successful,Doneil Henry,14.1,55.7,Vancouver,23.2,75.0,21.337760
4,2019,0,Successful,Maxime Crépeau,22.9,72.3,Vancouver,14.1,55.0,19.409534
5,2019,0,Successful,Doneil Henry,13.4,51.9,Vancouver,22.8,73.2,23.281967
6,2019,0,Successful,Maxime Crépeau,14.7,12.8,Vancouver,8.5,51.1,38.798582
7,2019,0,Unsuccessful,Erik Godoy,63.1,14.6,Vancouver,20.0,3.8,44.432533
8,2019,0,Successful,Nedum Onuoha,43.2,68.7,Salt Lake,38.0,77.6,10.307764
9,2019,0,Successful,Kyle Beckerman,32.3,95.0,Salt Lake,38.6,71.4,24.426420


In [103]:
dash_df = pd.DataFrame(
    columns = ["Player", "Date"]
)

In [104]:
dash_df["Player"] = touch_df["Player"]

In [105]:
dash_df.merge(touch_df["Date"])

,Player,Date


In [86]:
dash_df = touch_df.pivot_table(
    values = "Distance",
    index = ["Player", "Date"],
    aggfunc = "max"
).assign(
"Player" = touch_df["Player"]
)

SyntaxError: keyword can't be an expression (<ipython-input-86-5872a000e29e>, line 6)

In [81]:
dash_df[dash_df["Player"] == "A-Jay Leitch-Smith"]["Distance"].max()

SyntaxError: invalid syntax (<ipython-input-81-39e503f3ebc5>, line 1)

In [ ]:
dash_df = (
    touch_df.pivot_table(
        columns = ["Player"],
        #index=["Player", valami más],
        values="Date",
        #aggfunc="count",
    )
    .fillna(0)
    .assign(
        **{
            "Pass success rate": lambda touch_df: (touch_df.groupby(["Player","Date"])["Outcome"].count()/touch_df.groupby(["Player"])["Outcome"].count())
        }))

In [34]:
while True:
i = 0

dash_df["Indicator Name"] = 
 = touch_df.groupby(["Player","Date"])["Outcome"].count()/touch_df.groupby(["Player"])["Outcome"].count()

pandas.core.series.Series

játékos szinten évekre bontva:
- milyen sikerrátával passzolt - Pass success rate
- milyen hosszú volt a leghosszabb passza - longest pass distance
- milyen hosszú volt a legrövidebb passza - shortest pass distance
- milyen hosszú volt egy átlag passza - avarage pass distance
- melyik 10 percekben passzolt - pass intensive minutes

In [17]:
import pandas as pd
df = pd.read_csv('https://plotly.github.io/datasets/country_indicators.csv')

In [18]:
df

,Country Name,Indicator Name,Year,Value
0,Arab World,"Agriculture, value added (% of GDP)",1962,NaN
1,Arab World,CO2 emissions (metric tons per capita),1962,0.760996
2,Arab World,Domestic credit provided by financial sector (...,1962,18.168690
3,Arab World,Electric power consumption (kWh per capita),1962,NaN
4,Arab World,Energy use (kg of oil equivalent per capita),1962,NaN
5,Arab World,Exports of goods and services (% of GDP),1962,NaN
6,Arab World,"Fertility rate, total (births per woman)",1962,6.964454
7,Arab World,GDP growth (annual %),1962,NaN
8,Arab World,Imports of goods and services (% of GDP),1962,NaN
9,Arab World,"Industry, value added (% of GDP)",1962,NaN
